<a href="https://colab.research.google.com/github/Musab678/LDA/blob/main/use_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load the base model (4-bit quantized)
base_model = AutoModelForCausalLM.from_pretrained(
    "unsloth/deepseek-r1-distill-llama-8b-unsloth-bnb-4bit",
    device_map="auto",  # Automatically uses GPU if available
    trust_remote_code=True,
)

# Load the fine-tuned PEFT adapter
model = PeftModel.from_pretrained(base_model, "Musabjee/Business-assistant-thinking")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("unsloth/deepseek-r1-distill-llama-8b-unsloth-bnb-4bit")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a business expert with advanced knowledge in Accounting, Banks and Banking, Business, Business Communication, Career management, E-Business, Economics, Entrepreneurship, Finance, Financial Planning, Industries and Professions, Investments, Management and Leadership, Marketing and Sales, Real Estate, Stock Trading, Tax, Small Business, and Entrepreneurship.
Please answer the following Business-related question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [ ]:
prompt_style ="""Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a business expert with advanced knowledge in Accounting, Banks and Banking, Business, Business Communication, Career management, E-Business, Economics, Entrepreneurship, Finance, Financial Planning, Industries and Professions, Investments, Management and Leadership, Marketing and Sales, Real Estate, Stock Trading, Tax, Small Business, and Entrepreneurship.
Please answer the following Business-related question.


### Question:
{}

### Response:
<think>{}"""

In [ ]:
def generate_response(question):
    # Format the prompt
    prompt = prompt_style.format(question, "")

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move to GPU

    # Generate the response
    outputs = model.generate(
        **inputs,
        max_new_tokens=1200,  # Adjust based on desired response length
        temperature=0.7,      # Controls creativity (lower = more deterministic)
        top_p=0.95,           # Nucleus sampling threshold
        repetition_penalty=1.15,
    )

    # Decode and extract the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("### Response:")[1].strip()

# Example usage
question = "today is sunday yester is ?"
response = generate_response(question)
print(response)

<think>
Okay, so I'm trying to figure out what the correct answer is for this business-related question. The question says, "Today is Sunday yesterday is?" At first glance, it seems straightforward, but since it's a business scenario, maybe there's more to it than meets the eye. 

Let me break it down. Today is Sunday, which means yesterday was Saturday. But why would someone be asking about yesterday in a business context? Maybe they're scheduling meetings or events. If today is Sunday, yesterday was Saturday. So if the user needs to refer back to the previous day's information, they should look at Saturday's data or records.

I wonder if there's any nuance here related to time zones or specific business practices. Perhaps the user is working across different regions with varying time zones. However, without additional information, it's safe to assume that yesterday was simply Saturday. This might seem too simple, but sometimes questions can have hidden complexities, especially in int

In [ ]:
def is_valid_category(query):
    """Check if the query falls within the business categories."""
    business_keywords = [
        "accounting", "banking", "business", "career", "e-business", "economics",
        "entrepreneurship", "finance", "investments", "management", "marketing",
        "real estate", "stock trading", "tax", "small business"
    ]
    return any(keyword in query.lower() for keyword in business_keywords)

# Example usage
question = "what is differnec between B2B and B2C business"
if is_valid_category(question):
    response = generate_response(question)
    print(response)
else:
    print("I'm specialized in business-related topics. Please ask a business-related question.")

<think>
Okay, so I need to figure out the difference between B2B and B2C businesses. Hmm, let's start by breaking down what each acronym stands for. B2B probably means business-to-business, while B2C likely refers to business-to-consumer. That makes sense because "to" indicates the audience or customer type. But wait, I should make sure I'm not mixing up anything here. 

Thinking deeper, B2B involves selling products or services directly to other businesses. For example, companies providing software solutions to other businesses would be B2B. On the other hand, B2C is when businesses sell directly to individual consumers. Like a coffee shop selling lattes to customers—that's definitely B2C.

I wonder if there are key differences beyond just who they're selling to. Maybe in terms of marketing strategies? B2B might use more specialized advertising aimed at other businesses, such as trade shows or industry-specific publications. Meanwhile, B2C could lean more heavily on consumer-focused a